### Question 1: Use yfinance to Extract Stock Data

In [1]:
!pip install yfinance requests beautifulsoup4 lxml plotly 


[notice] A new release of pip is available: 24.3.1 -> 25.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [11]:
pip install selenium webdriver-manager


  Using cached python_dotenv-1.1.0-py3-none-any.whl.metadata (24 kB)
Using cached python_dotenv-1.1.0-py3-none-any.whl (20 kB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [19]:
!pip install nbformat



[notice] A new release of pip is available: 24.3.1 -> 25.1
[notice] To update, run: python.exe -m pip install --upgrade pip


# Question 1: Use yfinance to Extract Stock Data

In [12]:
import yfinance as yf

tesla = yf.Ticker("TSLA")
tesla_data = tesla.history(period="max")
tesla_data.reset_index(inplace=True)
tesla_data.head()


,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2010-06-29 00:00:00-04:00,1.266667,1.666667,1.169333,1.592667,281494500,0.0,0.0
1,2010-06-30 00:00:00-04:00,1.719333,2.028000,1.553333,1.588667,257806500,0.0,0.0
2,2010-07-01 00:00:00-04:00,1.666667,1.728000,1.351333,1.464000,123282000,0.0,0.0
3,2010-07-02 00:00:00-04:00,1.533333,1.540000,1.247333,1.280000,77097000,0.0,0.0
4,2010-07-06 00:00:00-04:00,1.333333,1.333333,1.055333,1.074000,103003500,0.0,0.0


# Question 2: Use Webscraping to Extract Tesla Revenue Data

In [13]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

url = "https://www.macrotrends.net/stocks/charts/TSLA/tesla/revenue"
html_data = requests.get(url).text
soup = BeautifulSoup(html_data, "html.parser")

# Find the correct table by identifying the table that has the words "Quarterly Revenue"
tables = soup.find_all("table")

tesla_revenue = pd.DataFrame(columns=["Date", "Revenue"])

for table in tables:
    if "Tesla Quarterly Revenue" in str(table):
        for row in table.tbody.find_all("tr"):
            cols = row.find_all("td")
            if len(cols) == 2:
                date = cols[0].text.strip()
                revenue = cols[1].text.strip().replace('$', '').replace(',', '')
                if revenue:
                    tesla_revenue = pd.concat([tesla_revenue, pd.DataFrame([[date, revenue]], columns=["Date", "Revenue"])])
        break

# Convert revenue to numeric
tesla_revenue["Revenue"] = pd.to_numeric(tesla_revenue["Revenue"])
tesla_revenue.head()


,Date,Revenue


### Question 3: Use yfinance to Extract Stock Data

In [14]:
gme = yf.Ticker("GME")
gme_data = gme.history(period="max")
gme_data.reset_index(inplace=True)
gme_data.head()


,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2002-02-13 00:00:00-05:00,1.620128,1.693350,1.603296,1.691666,76216000,0.0,0.0
1,2002-02-14 00:00:00-05:00,1.712707,1.716074,1.670626,1.683250,11021600,0.0,0.0
2,2002-02-15 00:00:00-05:00,1.683251,1.687459,1.658002,1.674834,8389600,0.0,0.0
3,2002-02-19 00:00:00-05:00,1.666418,1.666418,1.578047,1.607504,7410400,0.0,0.0
4,2002-02-20 00:00:00-05:00,1.615920,1.662210,1.603296,1.662210,6892800,0.0,0.0


### Question 4: Use Webscraping to Extract GME Revenue Data

In [15]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import pandas as pd

# Set up the WebDriver
options = webdriver.ChromeOptions()
options.add_argument("--headless")  # Run in headless mode
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# Fetch the webpage
url = "https://www.macrotrends.net/stocks/charts/TSLA/tesla/revenue"
driver.get(url)

# Wait for the page to load
driver.implicitly_wait(10)

table = driver.find_elements(By.TAG_NAME, "table")[1]

tesla_revenue = pd.DataFrame(columns=["Date", "Revenue"])


rows = table.find_elements(By.TAG_NAME, "tr")
for row in rows:
    cols = row.find_elements(By.TAG_NAME, "td")
    if len(cols) == 2:
        date = cols[0].text.strip()
        revenue = cols[1].text.strip().replace('$', '').replace(',', '')
        if revenue:
            tesla_revenue = pd.concat([tesla_revenue, pd.DataFrame([[date, revenue]], columns=["Date", "Revenue"])])


tesla_revenue["Revenue"] = pd.to_numeric(tesla_revenue["Revenue"])
print(tesla_revenue)

driver.quit()


          Date  Revenue
0   2025-03-31    19335
0   2024-12-31    25707
0   2024-09-30    25182
0   2024-06-30    25500
0   2024-03-31    21301
..         ...      ...
0   2010-09-30       31
0   2010-06-30       28
0   2010-03-31       21
0   2009-09-30       46
0   2009-06-30       27

[63 rows x 2 columns]


In [25]:
import nbformat
print(nbformat.__version__)  # should be >= 4.2.0


5.10.4


### Question 5: Plot Tesla Stock Graph

In [ ]:
import plotly.graph_objects as go
import plotly.io as pio


pio.renderers.default = "browser"

fig = go.Figure()
fig.add_trace(go.Scatter(x=tesla_data['Date'], y=tesla_data['Close'], name="Tesla Stock Price"))
fig.update_layout(title="Tesla Stock Price", xaxis_title="Date", yaxis_title="Stock Price ($)")
fig.show()

fig = go.Figure()
fig.add_trace(go.Scatter(x=tesla_revenue['Date'], y=tesla_revenue['Revenue'], name="Tesla Revenue"))
fig.update_layout(title="Tesla Revenue", xaxis_title="Date", yaxis_title="Revenue ($ Millions)")
fig.show()


### Question 6: Plot GameStop Stock Graph

In [29]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=gme_data['Date'], y=gme_data['Close'], name="GameStop Stock Price"))
fig.update_layout(title="GameStop Stock Price", xaxis_title="Date", yaxis_title="Stock Price ($)")
fig.show()

fig = go.Figure()
fig.add_trace(go.Scatter(x=gme_revenue['Date'], y=gme_revenue['Revenue'], name="GameStop Revenue"))
fig.update_layout(title="GameStop Revenue", xaxis_title="Date", yaxis_title="Revenue ($ Millions)")
fig.show()
